## ICcl Model

In [1]:
# Import the needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# To import these NEURON needs to be installed on device
from neuron import h
from neuron import gui

## Compartmental Model

In [2]:
# 0 for AMPA style input and 1 for NMDA style input
stim_type = 0

In [3]:
# Parameters for the morphology and passive membrane conductance
spine_diam = .5 # (microns)
spine_len = 20 # (microns)
spine_g_pas = .004 # passive conductance in spine (microsiemens) 

soma_diam = 20  # (microns)
soma_len = 15 # (microns)
soma_g_pas = .004 # (microsiemens)

# Parameter for the dendrite diameter. To have dendrites with different diameters, remove this parameter
dd = 5 # (microns)

dend1_diam = dd
dend2_diam = dd
dend3_diam = dd
dend4_diam = dd
dend5_diam = dd
dend_len = 100 # (microns)
dend_g_pas = .004 # (microsiemens)

# Number of Simulations
nsim = 100

#Parameters for Sound Input
weight_input = .5 # weight out front in equation that determines the interval of input
weight_in_gaussian = 2/1000 # weight within the exponent of the gaussion in the interval equation
max_interval = 100 # (ms)

# Frequencies are in kHz/1000 becuase we are working in ms
freq_in_1 = 4.5/1000 
freq_in_2 = 5.5/1000 
freq_pref_1 = 4.5/1000
freq_pref_2 = 5.5/1000

itd_in = np.zeros(nsim) # (ms)
itd_pref = 0 # (ms)

# Fill the ITD vector
x = np.arange(nsim)
for n in x:
        itd_in[n] = -200+n*4    

# Parameters for the NetCon and NetStim
num = 100 # Although this is large, it doesn't mean it will have 100 spikes
start = 0 # Start at 0 ms
noise = .1 # 0 for no noise. 1 for Poisson process 
weight = .03 # Weight for the Network Connection

#Declaring arrays in python to place the Vectors from the NEURON in
indpen = [] #For the independent sum of response1 and response2
simult = [] #Simultaneous input
response1 = [] #Response to input with freq_in_1
response2 = [] #Response to input with freq_in_2

if(stim_type == 0):
    #Loading the simulation file for AMPA style input
    h.load_file('Simple_Model_py_expsyn_v5_spine.hoc')
    
    # Fill the NEURON parameter with the sound parameters defined in python above
    h.w = weight_input 
    h.w2 = weight_in_gaussian
    h.f_i_1 = freq_in_1
    h.f_i_2 = freq_in_2
    h.f_p_1 = freq_pref_1
    h.f_p_2 = freq_pref_2
    h.itd_p = itd_pref
    h.max_int = max_interval

    # Fill the NEURON parameter with the NetCon parameters defined in python above
    h.number = num # Although this is large, it doesn't mean it will have 100 spikes
    h.start = start # Start at __ ms
    h.noise = noise #1 for Poisson Process, 0 for no uniform intervals 
    h.weight = weight # Weight of the Connection
    
    # Call custom NEURON functions to establish the morphology (these can be found in the hoc code)
    h.geom(soma_diam, soma_len, dend1_diam , dend2_diam, dend3_diam, dend4_diam, dend5_diam, dend_len)
    h.biophys(soma_g_pas,dend_g_pas)
    h.spines(spine_diam, spine_len,spine_g_pas)
    h.spines_soma(spine_diam,spine_len,spine_g_pas)
    
    # Fill the NEURON ITD vector
    for n in x:
        h.itd_i.x[n]= itd_in[n]
       
    # Call the custom NEURON function to run simulations (this may take a minute or two depending on resolution)
    h.run_simulation()
    
    # Take the results of the simulations and put them in the python vectors
    for n in x:
        response1.append(h.soma_response1.x[n])
        response2.append(h.soma_response2.x[n])
        indpen.append(h.soma_ind_combo.x[n])
        simult.append(h.soma_combined.x[n])

elif(stim_type == 1):
    #Loading the simulation file for NMDA style input
    h.load_file('Simple_Model_py_nmda_v5_spine.hoc')
    
    # Parameters for netcon. Since there are different input functions weights need to be rebalanced
    weight = .5
    
    # Fill the NEURON parameter with the sound parameters defined in python above
    h.w = weight_input
    h.w2 = weight_in_gaussian
    h.f_i_1 = freq_in_1
    h.f_i_2 = freq_in_2
    h.f_p_1 = freq_pref_1
    h.f_p_2 = freq_pref_2
    h.itd_p = itd_pref
    h.max_int = max_interval

    # Fill the NEURON parameter with the NetCon parameters defined in python above
    h.number = num # Although this is large, it doesn't mean it will have that many spikes
    h.start = start # Start at __ ms
    h.noise = noise #1 for Poisson Process, 0 for no noise
    h.weight = weight # Weight of the Connection
    
    # Call custom NEURON functions to establish the morphology (these can be found in the hoc code)
    h.geom(soma_diam, soma_len, dend1_diam , dend2_diam, dend3_diam, dend4_diam, dend5_diam, dend_len)
    h.biophys(soma_g_pas,dend_g_pas)
    h.spines(spine_diam, spine_len,spine_g_pas)
    h.spines_soma(spine_diam,spine_len,spine_g_pas)
    
    # Fill the NEURON ITD vector
    for n in x:
        h.itd_i.x[n]= itd_in[n]
       
    # Running the simulation
    h.run_simulation()
    
    # Take the results of the simulations and put them in the python vectors
    for n in x:
        response1.append(h.soma_response1.x[n])
        response2.append(h.soma_response2.x[n])
        indpen.append(h.soma_ind_combo.x[n])
        simult.append(h.soma_combined.x[n])

## Export Results

In [4]:
# Create two arrays to hold the data
# These are seperated into two because they are used in seperate visualizations
plot_data = [response1,response2,simult]
plot_data2 = [indpen,simult]

df = pd.DataFrame(data=plot_data)
df2 = pd.DataFrame(data=plot_data2)
df.to_csv('FILE NAME FOR plot_data') # This CSV will hold res1, res2, and simult
df2.to_csv('FILE NAME FOR plot_data2') # This CSV will hold indpen, simult